In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 📘 CityAI – Decisions sobre inversions en inmobles

Aquest notebook utilitza un escenari plausible de l'Ajuntament de Barcelona per aplicar l'algorisme de la motxilla en un problema real.

## 🧠 Objectius

Aprofundir en els algorismes vistos a teoria, i veure la seva aplicació en un cas real.

## 🧩 Context

L'Ajuntament de Barcelona disposa de 20 milions d'euros per invertir en projectes immobiliaris de millora urbana. Té una llista de possibles inversions.

|            Projecte         | Cost (M€) | Retorn economic (M€) | Impacte social (sobre 10) |
| ----------------------------|-----------|----------------------|------------------|
| Habitatge social al Raval           | 8  | 10 | 9|
| Reforma mercat de Sant Antoni       | 6  | 7  | 6|
| Centre cívic a Nou Barris           | 4  | 5  | 8|
| Rehabilitació edificis Eixample     | 10 | 12 | 5|
| Oficina d'atenció ciutadana a Sants | 3  | 3  | 7|
| Habitatge sostenible al Poblenou    | 7  | 9  | 8|

Considerarem que el valor de cada inversió és:

valor = 0.8 x retorn_economic + 0.2 x impacte_social


In [ ]:
# Per facilitar les proves, es proporcionen les dades inicials aquí mateix  

inversions = [
    ("Habitatge social al Raval", 8, 10, 9), 
    ("Reforma mercat de Sant Antoni", 6, 7, 6),
    ("Centre cívic a Nou Barris", 4, 5, 8),
    ("Rehabilitació edificis Eixample", 10, 12, 5),    
    ("Oficina d'atenció ciutadana a Sants", 3, 3, 7),
    ("Habitatge sostenible al Poblenou", 7, 9, 8)
]
pressupost = 20

## ✍️ Exercici 1: Implementa l'algorisme de la motxilla amb les dades del cas

Tenim un conjunt d'opcions (immobles) cadascuna amb un cost i un benefici esperat (econòmic, social o mixt). L'Ajuntament disposa d'un pressupost màxim i vol maximitzar el benefici total.

Podem fer el paral·lelisme amb la motxilla:
+ Pes => cost d'inversió
+ Valor => tendiment esperat
+ Capacitat màxima => Pressupost total

Adapta el codi de la motxilla amb el cas descrit i amb el valor ponderat descrit més amunt.

Quines són les inversions triades?

 Inversions triades amb pressupost de 20 M€

Escenari 1 – Ponderació 0.8 econòmic / 0.2 social
- Reforma mercat de Sant Antoni  
- Centre cívic a Nou Barris  
- Oficina d’atenció ciutadana a Sants  
- Habitatge sostenible al Poblenou  

**Cost total:** 20 M€  
**Valor combinat:** 25.0

In [4]:
from itertools import combinations

def motxilla_fb(
    nombre:int, capacitat:int, pes_cost:list[tuple[float, float]]
) -> (float, list[int]):
    """
    Tria la millor combinació d'objectes amb l'algoritme de la motxilla (força bruta)

    Parametres:
        :param nombre: nombre d'items possibles
        :param capacitat: capacitat de la motxilla (pressupost)
        :param pes_cost:
            llista de tuples del tipus [(pes, valor), (pes, valor), ...]

    Retorn:
        Tupla on el primer element és el valor total i el segon element és la
        combinació òptima d'objectes com a vector binari de longitud 'nombre'.
    """
    millor_valor = 0.0
    millor_combinacio = [0] * nombre

    # Prova totes les combinacions possibles
    indices = list(range(nombre))
    for r in range(nombre + 1):
        for combo in combinations(indices, r):
            pes_total = sum(pes_cost[i][0] for i in combo)
            if pes_total <= capacitat:
                valor_total = sum(pes_cost[i][1] for i in combo)
                if valor_total > millor_valor:
                    millor_valor = valor_total
                    millor_combinacio = [1 if i in combo else 0 for i in indices]

    return millor_valor, millor_combinacio


In [5]:
motxilla_fb(6, 20, [
    (8, 0.8 * 10 + 0.2 * 9), (6, 0.8 * 7 + 0.2 * 6), (48, 0.8 * 5 + 0.24 * 8), 
    (10, 0.8 * 12 + 0.2 * 5), (3, 0.8 * 3 + 0.2 * 7), (7, 0.8 * 9 + 0.2 * 8)
])

(23.200000000000003, [0, 0, 0, 1, 1, 1])

## ✍️ Exercici 2 – Discussió i reflexió

Què passaria si es donés més pes a l'impacte social que al retorn econòmic? per exemple 0.3 per retorn econòmic i 0.7 per retorn social.

Retoca l'algorisme o la crida i observa els resultats

In [6]:
pes_cost_30_70 = [
    (8, 9.3),   # Raval
    (6, 6.3),   # Sant Antoni
    (4, 7.1),   # Nou Barris
    (10, 7.1),  # Eixample
    (3, 5.4),   # Sants
    (7, 8.7)    # Poblenou
]

valor_30_70, seleccio_30_70 = motxilla_fb(6, 20, pes_cost_30_70)
print(valor_30_70, seleccio_30_70)


27.499999999999996 [0, 1, 1, 0, 1, 1]


## ✍️ Exercici 3 – Crea funcions auxiliars que 

1. Permetin dir el percentatge assignat a cada concepte (retorn econòmic i impacte social) i facin la crida corresponent a l'algorisme de la motxilla. 
1. Crein un diccionari amb la posició a la llista i el nom del projecte i quan l'algorisme retorni el resultat treguin un llistat dels projectes seleccionats.

In [10]:
def percentatge(ret_eco:float, imp_soc:float, llista:list[tuple[str, float, float, float]], capacitat:int=20) -> (float, list[int]):
    """
    Calcula la combinació òptima amb pesos personalitzats per retorn econòmic i impacte social.

    Parametres:
        :param ret_eco: pes del retorn econòmic (p.ex. 0.8)
        :param imp_soc: pes de l'impacte social (p.ex. 0.2)
        :param llista: llista d'inversions [(projecte, cost, retorn_econòmic, impacte_social), ...]
        :param capacitat: pressupost total disponible

    Retorn:
        (valor_màxim, combinació òptima com a vector binari)
    """
    pes_cost = []
    for (_, cost, r, s) in llista:
        valor = ret_eco * r + imp_soc * s
        pes_cost.append((cost, valor))

    return motxilla_fb(len(llista), capacitat, pes_cost)

# Definim les inversions
inversions = [
    ("Habitatge social al Raval", 8, 10, 9),
    ("Reforma mercat de Sant Antoni", 6, 7, 6),
    ("Centre cívic a Nou Barris", 4, 5, 8),
    ("Rehabilitació edificis Eixample", 10, 12, 5),
    ("Oficina d’atenció ciutadana a Sants", 3, 3, 7),
    ("Habitatge sostenible al Poblenou", 7, 9, 8)
]


valor, seleccio = percentatge(0.8, 0.2, inversions)
print("Valor òptim:", valor)
print("Selecció òptima:", seleccio)



Valor òptim: 25.000000000000004
Selecció òptima: [0, 1, 1, 0, 1, 1]


In [11]:
percentatge(0.8, 0.2, inversions)

(25.000000000000004, [0, 1, 1, 0, 1, 1])

In [13]:
def resultats(selec:list[int], inversions:list[tuple[str, float, float, float]]) -> list[str]:
    """
    Tradueix la combinació de projectes a la llista amb els noms dels projectes.

    Parametres:
        :param selec: 
            llista amb els projectes seleccionats (1) i no seleccionats (0)
        :param inversions: 
            llista de tuples del tipus 
            [(projecte, pes, retorn_economic, impacte_social), ...]

    Retorn:
        Llista amb els noms dels projectes que té la combinació introduïda.
    """
    seleccionats = []
    for i, flag in enumerate(selec):
        if flag == 1:
            seleccionats.append(inversions[i][0])  
    return seleccionats


In [14]:
resultats([1, 0, 0, 1, 0, 1], inversions)

['Habitatge social al Raval',
 'Rehabilitació edificis Eixample',
 'Habitatge sostenible al Poblenou']